<h1 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:20px; margin-bottom:10px;">بارگذاری، ادغام و بررسی ساختار داده‌ها</h1>

<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
در قدم اول، علاوه بر ایمپورت کتابخانه های ضروری مانند (pandas)، برای جلوگیری از هدر رفت زمان و منابع شبکه، مجموعه داده‌های آموزشی و نگاشت (Mapping) را به صورت مستقیم و از طریق کتابخانه gdown وارد می‌کنیم. 
بررسی‌های اولیه نشان می‌دهد که ستون‌های کلیدی مسئله، نظیر overall (متغیر هدف) و reviewText (متغیر مستقل)، فاقد داده‌های گم شده  هستند که این موضوع پایداری فاز مدل‌سازی را تضمین می‌کند.
</p>


In [10]:
import pandas as pd
import gdown
from IPython.display import display, HTML

In [2]:
u =f"https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW"
f ="train_data.csv"
gdown.download(u, f, quiet =False)


Downloading...
From (original): https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW
From (redirected): https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW&confirm=t&uuid=eda21791-6a1c-4f22-92cd-bb9a4ad3f7b2
To: /content/train_data.csv
100%|██████████| 635M/635M [00:06<00:00, 93.0MB/s]


'train_data.csv'

In [4]:
df =pd.read_csv(f, low_memory=False)
print(df.shape)
print(df.isnull().sum())
print(df.head())

(838944, 11)
overall                0
vote              647476
verified               0
reviewTime             0
reviewerID             0
asin                   0
style             348331
reviewerName         227
reviewText             0
summary               76
unixReviewTime         0
dtype: int64
   overall vote  verified  reviewTime      reviewerID        asin style  \
0        2  NaN     False  2016-11-11   A2OSUEZJIN7BI  0511189877   NaN   
1        5  NaN      True  2016-06-06  A2NETQRG6JHIG7  0511189877   NaN   
2        4  NaN      True  2016-03-10  A12JHGROAX49G7  0511189877   NaN   
3        5  NaN      True  2016-01-14  A1KV65E2TMMG6F  0511189877   NaN   
4        5  NaN      True  2016-10-20  A280POPEWI0NSA  0594459451   NaN   

    reviewerName                                         reviewText  \
0          Chris  I have an older URC-WR7 remote and thought thi...   
1   Qrysta White  First time I've EVER had a remote that needed ...   
2        Linwood  Got them and only

In [5]:
i ="1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h"
u =f"https://drive.google.com/uc?id={i}"
f2 ="mapping.csv"
gdown.download(u, f2, quiet =False)

Downloading...
From: https://drive.google.com/uc?id=1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h
To: /content/mapping.csv
100%|██████████| 97.3M/97.3M [00:00<00:00, 118MB/s]


'mapping.csv'

In [6]:
dm =pd.read_csv(f2, low_memory =False)
print(dm.shape)
print(dm.isnull().sum())
print(dm.head())

(786445, 3)
asin        0
title      19
brand    5498
dtype: int64
         asin                                              title  \
0  0011300000  Genuine Geovision 1 Channel 3rd Party NVR IP S...   
1  0043396828  Books "Handbook of Astronomical Image Processi...   
2  0060009810                                     One Hot Summer   
3  0060219602  Hurray for Hattie Rabbit: Story and pictures (...   
4  0060786817                     sex.lies.murder.fame.: A Novel   

                                          brand  
0                                     GeoVision  
1                                  33 Books Co.  
2  Visit Amazon's Carolina Garcia Aguilera Page  
3           Visit Amazon's Dick Gackenbach Page  
4              Visit Amazon's Lolita Files Page  


<h1 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:20px; margin-bottom:10px;">پاکسازی و استانداردسازی</h1>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
نام بسیاری از برندها با پیشوندها و پسوندهای اضافی نظیر "Visit Amazon's" و "Page" یا "Store" ذخیره شده‌اند. عدم اصلاح این نویزها باعث می‌شود تا موتور Groupby یک برند واحد را به عنوان چندین موجودیت مجزا در نظر بگیرد و محاسبات آماری دچار انحراف شدید شود. برای حل این مشکل، از Regex استفاده کردیم.
</p>


In [7]:
dm['brand'] =dm['brand'].str.replace(r"^Visit Amazon's\s+", "", regex =True)
dm['brand'] =dm['brand'].str.replace(r"\s+Page$", "", regex =True)
dm['brand'] =dm['brand'].str.replace(r"\s+Store$", "", regex =True)

<h1 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:20px; margin-bottom:10px;">استخراج ۱۰ محصول برتر با امتیاز کامل (۵ ستاره)</h1>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
ابتدا یک ماسک بولی روی داده‌های اصلی اعمال شد تا تنها نظرات ۵ ستاره در حافظه باقی بمانند. سپس عملیات گروه‌بندی بر اساس شناسه‌ی محصول (asin) صورت گرفت و ۱۰ محصول برتر استخراج شدند.  در نهایت، برای به دست آوردن نام و برند این محصولات، تنها همین ۱۰ رکوردِ فیلترشده با جدول نگاشت به روش Left Join ادغام شدند.
</p>


In [11]:
df_5star =df[df['overall'] ==5]
grp_prod =df_5star.groupby('asin').size().reset_index(name ='count_5')
top_prod =grp_prod.sort_values('count_5', ascending =False).head(10)

merg_q5 =pd.merge(top_prod, dm, on ='asin', how ='left')
q5 =merg_q5[['brand', 'title', 'count_5']]
h5 ="<h3>--- Q5: Top 10 Products with 5 Stars ---</h3>"
display(HTML(h5))
display(q5)

,brand,title,count_5
0,Cambridge Soundworks,OontZ Angle 3 Enhanced Stereo Edition IPX5 Spl...,1163
1,NETGEAR,NETGEAR N300 WiFi Range Extender (EX2700),659
2,Roku,Roku Streaming Stick (3600R) - HD Streaming Pl...,600
3,StarTech,StarTech USB 2.0 to SATA IDE Adapter (USB2SATA...,598
4,Logitech,Logitech M570 Wireless Trackball Mouse &ndash;...,552
5,Samsung,Samsung 850 EVO 500GB 2.5-Inch SATA III Intern...,529
6,Asus,ASUS Tri-Band Gigabit (AC3200) WiFi Router (Up...,471
7,VideoSecu,VideoSecu ML531BE TV Wall Mount for Most 27&qu...,448
8,"Arlo Technologies, Inc",Arlo - Wireless Home Security Camera System | ...,433
9,Samsung,Samsung 850 EVO 250GB 2.5-Inch SATA III Intern...,431


<h2 style="direction:rtl; text-align:right; font-family:'Vazir', 'Tahoma', sans-serif; margin-top:15px; margin-bottom:10px;">ارزیابی میانگین رضایت برای ۱۰ برند پربازدید</h2>
<p style="direction:rtl; text-align:justify; font-family:'Vazir', 'Tahoma', sans-serif; margin-bottom:12px;">
پس از ادغام دو جدول، ابتدا گروه‌بندی اول برای استخراج ۱۰ برندی که بیشترین حجم نظرات را به خود اختصاص داده‌اند انجام شد. نام این برندها در یک لیست ذخیره گردید. سپس با استفاده از متد isin، داده‌های ترکیبی فیلتر شدند تا تنها شامل این ۱۰ غول تجاری باشند. در نهایت، با گروه‌بندی مجدد روی این داده‌های فیلترشده، میانگین امتیاز  به دست آمد.
</p>


In [12]:
merg_q6 =pd.merge(df[['asin', 'overall']], dm[['asin', 'brand']], on ='asin', how ='inner')
grp_brand =merg_q6.groupby('brand').size().reset_index(name ='total_rev')
list_top10 =grp_brand.sort_values('total_rev', ascending =False).head(10)['brand'].tolist()
filt_q6 =merg_q6[merg_q6['brand'].isin(list_top10)]
q6 =filt_q6.groupby('brand')['overall'].mean().reset_index(name ='mean_rating')
q6 =q6.sort_values('mean_rating', ascending =False).reset_index(drop =True)
h6 ="<h3>--- Q6: Top 10 Brands by Review Count and Mean Rating ---</h3>"
display(HTML(h6))
display(q6)

,brand,mean_rating
0,Anker,4.244320
1,AmazonBasics,4.201468
2,Sabrent,4.028005
3,SanDisk,3.958810
4,Logitech,3.954061
5,Samsung,3.941268
6,Sony,3.938726
7,TP-LINK,3.928681
8,Asus,3.855550
9,NETGEAR,3.836274
